- Distance hist 500 [clelia not sure if smart idea, so do this to test the idea]
    - [ ] 10 runs MCM 0 for 500 samples, overlap co- ocurrence matrices, calculate expected number icc and cluster to this amount
    - [ ] Then use new-found “average mcm”.
    - [ ] From all samples used, take union of unique elements (so exclude the ones we did not see and remove duplicates) and recalculate counts and Len data
    - [ ] Now compare the distance hist we get for 500 with the one from a single run 5k -> compare against all digits and single some out like last time
    - [ ] Goal: want to know if this average mcm is really needed instead of just using a single run one

In [250]:
import numpy as np
import os
import json
import sys
import matplotlib.pyplot as plt
import skimage

sys.path.append("../")
import src.plot as myplot
import seaborn as sns

import sklearn

In [251]:
import importlib
import src.plot as myplot
importlib.reload(myplot)

<module 'src.plot' from '/Users/paulhosek/PycharmProjects/mcm/MCM_classifier/Classifier_1/0_classifier/paper_plots/../src/plot.py'>

In [252]:

import importlib
import src.paper_utils as utils
importlib.reload(utils)

<module 'src.paper_utils' from '/Users/paulhosek/PycharmProjects/mcm/MCM_classifier/Classifier_1/0_classifier/paper_plots/../src/paper_utils.py'>

In [253]:
sample_sizes = [500, 5000]
counts_samplesizes, mcms_samplesizes = utils.load_counts_mcm(sample_sizes, letter="", path_format="../Output/sample_sizes")


In [254]:
mcm_digit = 0
mcms_500 = mcms_samplesizes[sample_sizes.index(500)]
nr_runs = len(mcms_500)

coo_500 = np.array([myplot.create_cooccurance_matrix(mcms_500[i][mcm_digit]) for i in range(nr_runs)],dtype=int)



In [255]:
import scipy.cluster.hierarchy as sch
from scipy.spatial.distance import pdist
import numpy as np

mean_nr_icc = np.mean([len(mcm[mcm_digit]) for mcm in mcms_500])

m = coo_500.sum(axis=0)
linkage = sch.linkage(m, method="ward")
labels = sch.fcluster(linkage, t=mean_nr_icc, criterion="maxclust")
sorted_indices = np.argsort(labels)
sorted_m = m[sorted_indices[:,None],sorted_indices]


In [256]:
avg_mcm = labels.reshape((11,11))

In [257]:
avg_mcm.max()

31

In [258]:
# show training set and construct counts
# get training set for each 500 using the seed, get the union without duplicates
# construct counts

# utils.recreate_dataset("",0,500,seed=s, fname_format = "train-images-unlabled-{}.dat",fname_start="train-")
# utils.subsample_data(500, all_data_path="../INPUT_all/data", input_data_path="../INPUT/data", seed=42,fname_start = "train-")
samp = np.array([utils.recreate_dataset("",mcm_digit, 500, seed = s+1,fname_format= "train-images-unlabeled-{}.dat", fname_start="train-", all_data_path="../INPUT_all/data/", input_data_path = "../INPUT/data/") for s in range(5)])
samp = np.unique(samp)

In [259]:


def partition_to_str(mcm):
    """Take a partition map labeling 11x11 array and return the original string representation"""
    nr_icc = mcm.max()
    out = []

    for icc in range(nr_icc):
        idx = np.argwhere(mcm.flatten()==icc).flatten()
        x = np.zeros(121,dtype=int)
        x[idx] = 1
        out.append("".join(map(str, x)))
    return np.array(out, dtype=str)


In [260]:
samp

array(['0000000000000000000000000000000000000000111100001100000001100001000110000100001000011000011111000000000000000000000000000',
       '0000000000000000000000000000000000000011111000011110110001100000110110000111001100111000011111000000000000000000000000000',
       '0000000000000000000000000000000000000100010000100000010010000000010100000000100111111110000000000000000000000000000000000',
       ...,
       '0011000000001111110000011100110000110000110001000000100011000000100011000001000011000001000011100110000011111000000000100',
       '0100000000000000111000000111111000001100011000110000110001100001100110000011001100000110011100011000011111100000000000000',
       '1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111'],
      dtype='<U121')

In [261]:
(avg_mcm.flatten()==icc)

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False,  True,  True,
        True, False, False, False])

In [262]:
icc = 1
samp_gen = np.genfromtxt(samp, delimiter=1, dtype=int)
n_variables = avg_mcm.max()
Counts = []
for icc in range(1, n_variables+1):
    idc = np.argwhere(avg_mcm.flatten()==icc).flatten()
    p_count = np.zeros(2**len(idc))
    icc_strings = [int("".join([str(s) for s in state]), 2) for state in samp_gen[:,idc]]

    u,c = np.unique(icc_strings, return_counts=True)

    p_count[u] = c  
    Counts.append(list(p_count))


In [263]:
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.cluster import AgglomerativeClustering

# co_matrix = coo_500



# distance_matrix = 1 - co_matrix / co_matrix.max() 
# model = AgglomerativeClustering(n_clusters=5, affinity="euclidean", linkage="ward")
# model.fit(distance_matrix)
# cluster_labels = model.labels_

# order_label = np.argsort(cluster_labels) # ties not handled well
# reordered_matrix = co_matrix[order_label[:, None], order_label]

# # plt.imshow(reordered_matrix)
